In [57]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import os
import glob
import skimage.io
import joblib
from PIL import Image

In [58]:
from labelMaker import labelMaker

In [59]:
arr = np.ones((10, 5, 5, 200, 200, 3))

In [60]:
arr[0, 0, 0, :, :, :].shape

(200, 200, 3)

### Build dataset from data team's format

In [62]:
def build_dataset_from_arr(data_file, puzzle_dims=(5,5)):
    
    im_list = []
    y_list = []
    
    data_arr = joblib.load(data_file)

    for puzz_idx in range(data_arr.shape[0]):
        
        
        for i in range(data_arr.shape[1]):
            for j in range(data_arr.shape[2]):
                    
                im_list.append((data_arr[puzz_idx, i, j, :, :, :], (i,j)))
                
        
        X_list = []
        for i in range(len(im_list)):
            for j in range(len(im_list)):
                
                if not i == j:
                    X_list.append(
                        np.reshape(
                            np.concatenate(
                                (im_list[i][0], im_list[j][0])
                            ),
                            (data_arr.shape[3], data_arr.shape[4], 6)
                        )
                    )
                    y_list.append(np.array(labelMaker(im_list[i][1][0],im_list[j][1][0],im_list[i][1][1],im_list[j][1][1])))

                             
    
    full_X = np.empty((len(X_list), *X_list[0].shape))
    print('DEBUG: '+ str(full_X.shape[0]) + ' ' + str(full_X.shape[1]) + ' ' + str(full_X.shape[2]) + ' ' + str(full_X.shape[3]))
    full_y = np.empty((len(y_list), 5))
    
    for i in range(len(X_list)):
        #print(X_list[i])
        full_X[i] = X_list[i]
        #print(y_list[i].tolist())
        full_y[i] = y_list[i].tolist()
        
    
    return full_X, full_y

In [4]:
data_path = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\allImagesNumpy'

In [39]:
X, y = build_dataset_from_arr(data_path)

DEBUG: 71600 100 100 6


In [25]:
sample_X, sample_y = X[:100], y[:100]

In [26]:
sample_X.shape

(100, 100, 100, 6)

In [40]:
def preprocess_norm(X):
    X = X / 255.0
    
    return X

In [41]:
def equalize_labels(X, y, pct_connected=0.5):
    print(y)
    print(np.where(y[:,4] == 0))
    adj_X = X[np.where(y[:,4] == 0)]
    adj_y = y[np.where(y[:,4] == 0)]
    
    print(adj_X.shape)
    
    n_adj_X = X[np.where(y[:,4] == 1)]
    n_adj_y = y[np.where(y[:,4] == 1)]
    
    total_adj = adj_X.shape[0]

    idx_mask = np.random.choice(np.arange(n_adj_X.shape[0]), int((total_adj/pct_connected)-total_adj) , replace=False)
    
    X = np.concatenate((n_adj_X[idx_mask], adj_X))
    y = np.concatenate((n_adj_y[idx_mask], adj_y))
    
    return X, y
    

In [38]:
X = preprocess_norm(X)

In [42]:
X, y = equalize_labels(X, y)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
(array([], dtype=int64),)
(0, 100, 100, 6)


In [19]:
X.shape

(71600, 100, 100, 6)

In [15]:
y.shape

(0, 5)

In [12]:
save_path_X = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\preprocessedData_X'
save_path_y = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\preprocessedData_y'

In [13]:
joblib.dump(X, save_path_X, compress=0)
joblib.dump(y, save_path_y, compress=0)

['C:\\machine learning workspace\\final project\\MachineLearningTeamProject\\Model\\preprocessedData_y']

### Simple Model Playground

In [132]:
y

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]])

In [109]:
def build_CNN_w_pooling(X):    
    model = Sequential()

    model.add(Conv2D(32,(3,3),input_shape=X.shape[1:] ,activation = 'relu',padding='same',))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [111]:
model = build_CNN_w_pooling(X)

2022-05-15 21:27:15.197878: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [112]:
model.fit(X, y , epochs=30, batch_size=32, validation_split=0.1)

Epoch 1/30
17/17 [==============================] - 24s 1s/step - loss: 120.6463 - accuracy: 0.7481 - val_loss: 1.0253 - val_accuracy: 0.8500
Epoch 2/30
17/17 [==============================] - 24s 1s/step - loss: 0.7394 - accuracy: 0.8593 - val_loss: 1.1452 - val_accuracy: 0.8500
Epoch 3/30
17/17 [==============================] - 22s 1s/step - loss: 0.7084 - accuracy: 0.8685 - val_loss: 1.1197 - val_accuracy: 0.8500
Epoch 4/30
17/17 [==============================] - 22s 1s/step - loss: 0.6860 - accuracy: 0.8685 - val_loss: 0.7028 - val_accuracy: 0.8500
Epoch 5/30
17/17 [==============================] - 23s 1s/step - loss: 0.6620 - accuracy: 0.8685 - val_loss: 0.6723 - val_accuracy: 0.8500
Epoch 6/30
17/17 [==============================] - 24s 1s/step - loss: 0.6649 - accuracy: 0.8685 - val_loss: 0.8269 - val_accuracy: 0.8500
Epoch 7/30
17/17 [==============================] - 24s 1s/step - loss: 0.6653 - accuracy: 0.8685 - val_loss: 0.6992 - val_accuracy: 0.8500
Epoch 8/30


KeyboardInterrupt: 

In [113]:
model.predict(X)

array([[0.12516342, 0.10846931, 0.11553416, 0.10564344, 0.5451897 ],
       [0.09151299, 0.07960231, 0.07611708, 0.07968806, 0.67307955],
       [0.08452865, 0.06726538, 0.05977225, 0.06250986, 0.7259239 ],
       ...,
       [0.10909431, 0.10055214, 0.1113238 , 0.11079023, 0.5682395 ],
       [0.09062684, 0.08424608, 0.08398275, 0.09016786, 0.6509765 ],
       [0.10043259, 0.09283187, 0.09430876, 0.10005014, 0.6123767 ]],
      dtype=float32)

In [114]:
import matplotlib.pyplot as plt

In [120]:
X[0,:,:,:3].shape

(200, 200, 3)

In [142]:
X[0,:,:,:3]

array([[[ 96.,  89.,  63.],
        [121., 114.,  88.],
        [139., 132., 104.],
        ...,
        [185.,  66.,  36.],
        [178.,  65.,  36.],
        [171.,  63.,  31.]],

       [[ 99.,  92.,  66.],
        [115., 107.,  81.],
        [132., 123.,  96.],
        ...,
        [186.,  66.,  32.],
        [179.,  66.,  32.],
        [173.,  62.,  28.]],

       [[112., 102.,  77.],
        [116., 104.,  78.],
        [127., 116.,  88.],
        ...,
        [186.,  64.,  27.],
        [182.,  65.,  28.],
        [177.,  64.,  26.]],

       ...,

       [[ 40.,  47.,  39.],
        [ 40.,  47.,  39.],
        [ 38.,  45.,  37.],
        ...,
        [ 58.,  61.,  54.],
        [ 59.,  62.,  55.],
        [ 59.,  61.,  56.]],

       [[ 42.,  49.,  41.],
        [ 43.,  50.,  42.],
        [ 40.,  47.,  39.],
        ...,
        [ 51.,  52.,  46.],
        [ 53.,  56.,  49.],
        [ 54.,  56.,  51.]],

       [[ 42.,  49.,  41.],
        [ 42.,  49.,  41.],
        [ 40.,  

In [10]:
plt.imshow(X[0,:,:,3:])

NameError: name 'plt' is not defined